Try training the n-grams with more training data.

# Imports

In [ ]:
# Import libraries
import os,random,math,nltk,csv,re,string,gensim
from google.colab import drive
import pandas as pd, csv
from nltk import word_tokenize as tokenize
nltk.download('punkt')
from scipy import spatial
import numpy as np
import matplotlib.pyplot as plt

# Data Preparation

In [ ]:
# Mount google drive
drive.mount('/content/drive')
# Assign the parent directory
parentdir = "/content/drive/My Drive/lab2resources/sentence-completion"
# Assign the training directory
trainingdir=os.path.join(parentdir,"Holmes_Training_Data")

# Function for splitting training and testing data
def get_training_testing(training_dir=trainingdir,split=0.5):
  filenames=os.listdir(training_dir)
  n=len(filenames)
  print("There are {} files in the training directory: {}".format(n,training_dir))
  #random.seed(53)  #if you want the same random split every time
  random.shuffle(filenames)
  index=int(n*split)
  return(filenames[:index],filenames[index:])

training,testing=get_training_testing(trainingdir)

%config IPCompleter.greedy=True

In [ ]:
import os
len(os.listdir(trainingdir))

In [ ]:
# Read the question and answer files
questions=os.path.join(parentdir,"testing_data.csv")
answers=os.path.join(parentdir,"test_answer.csv")

# Visualize the questions
with open(questions) as instream:
    csvreader=csv.reader(instream)
    lines=list(csvreader)
qs_df=pd.DataFrame(lines[1:],columns=lines[0])
qs_df.head()

In [ ]:
# Visualize the answers
with open(answers) as instream:
    csvreader=csv.reader(instream)
    lines=list(csvreader)

# Store the answers as a list
answers_list=[item[1] for item in lines[1:]]
# Add to the question dataframe
qs_df['answer']=answers_list
qs_df.head()

# N-gram Language Model

In [ ]:
class language_model():
  
  # Train when initialised
  def __init__(self,trainingdir=trainingdir,files=[],discount=0.75):
    self.discount=discount
    self.training_dir=trainingdir
    self.files=files
    self.train()
  
  # Function for training
  def train(self):

    # Dictionaries for each unigram, bigram, and trigram model
    self.unigram={}
    self.bigram={}
    self.trigram={}

    ''' Read the questions and create n-gram dictionaries, make 'unknown' tokens, 
    apply absolute discounting and convert to probability. '''
    self._processfiles()
    self._make_unknowns()
    self._discount()
    self._convert_to_probs()
      
  # Function for processing a line (question)
  def _processline(self,line):

    # Tokenise the question and add 'start' and 'end' tokens at each end.
    tokens=["__START"]+tokenize(line)+["__END"]
    # Previous point for bigram and trigram (w_(n-1))
    previous="__END"
    # Second previous point for trigram (w_(n-2))
    previous2="__END"

    # Add counts for each word to the models
    for token in tokens:
      # Add the unigram count of the word
      self.unigram[token]=self.unigram.get(token,0)+1
      
      # Add the bigram count of the word given its previous word
      # {w1:(w2:n)}
      current=self.bigram.get(previous,{})
      current[token]=current.get(token,0)+1
      self.bigram[previous]=current

      # Add the trigram count of the word given its two previous words
      # An if statement that allows to start from the second word of the line.
      # {w1:(w2:(w3:n))}
      if previous != "__END":
        current=self.trigram.get(previous2,{})
        current2=current.get(previous,{})
        current2[token]=current2.get(token,0)+1
        self.trigram[previous2]=current
        self.trigram[previous2][previous]=current2

      # Move the window to the right
      previous2=previous
      previous=token

  # Function for processing a file
  def _processfiles(self):
    for afile in self.files:
      print("Processing {}".format(afile))
      try:
        with open(os.path.join(self.training_dir,afile)) as instream:
          for line in instream:
            line=line.rstrip()
            if len(line)>0:
              self._processline(line)
      except UnicodeDecodeError:
        print("UnicodeDecodeError processing {}: ignoring rest of file".format(afile))
    
  # Function for converting the counts of words to probability distributions       
  def _convert_to_probs(self):
    
    # Get the unigram probability -> count of the word/count of all words
    self.unigram={k:v/sum(self.unigram.values()) for (k,v) in self.unigram.items()}
    # Get the bigram probability -> count of the previous and current word seen together/count of the previous word
    self.bigram={key:{k:v/sum(adict.values()) for (k,v) in adict.items()} for (key,adict) in self.bigram.items()}
    self.trigram={key2:{key:{k:v/sum(adict.values()) for (k,v) in adict.items()} for (key,adict) in dicts.items()} for (key2, dicts) in self.trigram.items()}
  
  # Function for getting the probability of words 
  def get_prob(self,token,context="",method="unigram"):
    
    # Get unigram probability of the word, get one of the unknown token if not present.
    if method=="unigram":
      return self.unigram.get(token,self.unigram.get("__UNK",0))

    # Get probability of a word and its context word seen together
    elif method=="bigram":
      # Get the bigram probability of the word and context word, get unknown tokens if not present.
      bigram=self.bigram.get(context[-1],self.bigram.get("__UNK",{}))
      big_p=bigram.get(token,bigram.get("__UNK",0))

      # Calculate the reserved probability mass according to the word's unigram probability.
      lmbda=bigram["__DISCOUNT"]
      uni_p=self.unigram.get(token,self.unigram.get("__UNK",0))

      # Calculate the final probability
      p=big_p+lmbda*uni_p
      return p

    # Get probability of a word and its past two words seen together
    elif method=='trigram':
      trigram=self.trigram.get(context[-2],self.trigram.get("__UNK",{}))
      trigram2=trigram.get(context[-1],trigram.get("__UNK",{}))
      tri_p=trigram2.get(token,trigram2.get("__UNK",0))
      # Calculate the reserved probability
      try:
        lmbda=trigram2["__DISCOUNT"]
      except:
        lmbda=self.discount
      uni_p=self.unigram.get(token,self.unigram.get("__UNK",0))
      p=tri_p+lmbda*uni_p
      return p
    return 'a'
  
  # Function for computing the probability of a line and the number of words in the line
  def compute_prob_line(self,line,method="unigram"):
    
    tokens=["__START"]+tokenize(line)+["__END"]
    acc=0

    # If trigram, starts from the third word of the line.
    if method == 'trigram':
      for i,token in enumerate(tokens[2:]):
        # Get the sum of trigram probabilities of the words in the line
        result=self.get_prob(token,tokens[i:i+2],method)
        # To prevent math domain error, add the value only if it's non-zero.
        if result != 0:
          acc+=math.log(result)
      return acc,len(tokens[2:])
    else:
      # If bigram, starts from the second word of the line.
      for i,token in enumerate(tokens[1:]):
        # Get the sum of bigram probabilities
        result=self.get_prob(token,tokens[i:i+1],method)
        if result != 0:
          acc+=math.log(result)
      return acc,len(tokens[1:])

  # Function for computing the probability and length of a corpus in a file
  def compute_probability(self,filenames=[],method="unigram"):
    if filenames==[]:
      filenames=self.files
    total_p=0
    total_N=0
    # Process the corpora in the files
    for i,afile in enumerate(filenames):
      print("Processing file {}:{}".format(i,afile))
      try:
        with open(os.path.join(self.training_dir,afile)) as instream:
          for line in instream:
            line=line.rstrip()
            if len(line)>0:
              p,N=self.compute_prob_line(line,method=method)
              total_p+=p
              total_N+=N
      except UnicodeDecodeError:
        print("UnicodeDecodeError processing file {}: ignoring rest of file".format(afile))
    return total_p,total_N

  # Function for calculating the perplexity of a model
  def compute_perplexity(self,filenames=[],method="unigram"):

    # Compute the proability and length of the corpus in the given file
    p,N=self.compute_probability(filenames=filenames,method=method)
    # Apply the formula
    pp=math.exp(-p/N)
    return pp 
  
  # Function for replacing the words seen less than specific time with an unknown token.
  def _make_unknowns(self,known=2):

    # Create unknowns for unigram -> when the view count of a word is less than the given parameter, replace it with unknown.
    # Increment the view count of unknown if already exists.
    for (k,v) in list(self.unigram.items()):
      if v<known:
        del self.unigram[k]
        self.unigram["__UNK"]=self.unigram.get("__UNK",0)+v

    # Create unknowns for bigram -> Look at each word in bigram and replace with unknown if not present in the dict.
    isknown=0
    for (k,adict) in list(self.bigram.items()):
      for (kk,v) in list(adict.items()):
        isknown=self.unigram.get(kk,0)
        # If the second word of the bigram hasn't seen before, replace with unknown and delete the original.
        if isknown==0:
          adict["__UNK"]=adict.get("__UNK",0)+v
          del adict[kk]
      isknown=self.unigram.get(k,0)
      # If the first word hasn't seen, replace it with unknown and assign the second word to it.
      if isknown==0:
        del self.bigram[k]
        current=self.bigram.get("__UNK",{})
        current.update(adict)
        self.bigram["__UNK"]=current
      # If the word has seen, assign the second word to the word. (Do this in case the second word was replaced to unknown)
      else:
        self.bigram[k]=adict

    # Create unknowns for trigram -> The same process as the bigram but also checks the third word
    isknown=0
    for (k,dicts) in list(self.trigram.items()):
      for (kk,adict) in list(dicts.items()):
        for (kkk,v) in list(adict.items()):
          isknown=self.unigram.get(kkk,0)
          if isknown==0:
            adict["__UNK"]=adict.get("__UNK",0)+v
            del adict[kkk]

        isknown=self.unigram.get(kk,0)
        if isknown==0:
          del dicts[kk]
          current=self.trigram[k].get("__UNK",{})
          current.update(adict)
          self.trigram[k]["__UNK"]=current
        else:
          self.trigram[k][kk]=adict

      isknown=self.unigram.get(k,0)
      if isknown==0:
        del self.trigram[k]
        current=self.trigram.get("__UNK",{})
        current.update(dicts)
        self.trigram["__UNK"]=current
      else:
        self.trigram[k]=dicts
              
  # Function for applying absolute discount
  # Discount amount set to 0.75
  def _discount(self,discount=0.75):
    self.discount=discount
    #discount each bigram and trigram count by a small fixed amount
    self.bigram={k:{kk:value-discount for (kk,value) in adict.items()} for (k,adict) in self.bigram.items()}
    self.trigram={k:{kk:{kkk:value-discount for (kkk,value) in adict.items()} for (kk,adict) in dicts.items()} for (k,dicts) in self.trigram.items()}
    
    # For each word, store the total amount of the discount so that the total is the same.
    # For bigram, just reserve the probability mass according to the first word.
    for k in self.bigram.keys():
        lamb=len(self.bigram[k])
        self.bigram[k]["__DISCOUNT"]=lamb*discount

    # For trigram, reserve the probability mass according to the second word.
    for k in self.trigram.keys():
      adict = self.trigram[k]
      for kk in adict:
        lamb=len(adict[kk])
        self.trigram[k][kk]["__DISCOUNT"]=lamb*discount

In [ ]:
# Create filesets with the training and testing data
MAX_FILES=100
filesets={"training":training[:MAX_FILES],"testing":testing[:MAX_FILES]}

# Train the language model with the three methods
mylm=language_model(files=filesets["training"])
methods=["unigram","bigram",'trigram']

# Compute perplexity of the training and testing data using each model and store the results for plotting
perplexities={}
for f,names in list(filesets.items()):
  perplexities[f]={}
  for m in methods:
    p=mylm.compute_perplexity(filenames=names,method=m)
    print("Perplexity on {} with {} method is {}".format(f,m,p))
    perplexities[f][m]=p

In [ ]:
# The function for plotting perplexity on a bar chart
def autolabel(rects):
  for rect in rects:
    height=rect.get_height()
    # Decide the location where the value for each bar is presented 
    location=height
    # If the value is negative, place the value right above the x-axis
    if height<0:
      location=0
    # Annotate the value for each bar
    ax.annotate('{:.2f}'.format(height),
                xy=(rect.get_x() + rect.get_width() / 2, location),
                xytext=(0, 8),  # 3 points vertical offset
                textcoords="offset points",
                ha='center', va='bottom')

In [ ]:
# Plot the results
labels=['1 month', '6 months', '1 year']

x = np.arange(len(labels))  # the label locations
width=0.4  # the width of the bars

fig, ax = plt.subplots()
rects1=ax.bar(x - width/2, perplexities['training'].values(), width, label='training')
rects2=ax.bar(x + width/2, perplexities['testing'].values(), width, label='testing')
ax.set_ylabel('Perplexity')
ax.set_title('The perplexity of each model on training and testing data with 100 training documents')
ax.set_xticks(x)
ax.set_xticklabels(methods)
ax.set_ylim((0,800))
ax.legend()

autolabel(rects1)
autolabel(rects2)

plt.show()

In [ ]:
# Class for word similarity method using word embeddings
class word_embedding:

  # Load pre-trained word2vec
  def __init__(self):
    embedding_file='/content/drive/MyDrive/GoogleNews-vectors-negative300.bin'
    self.embeddings=gensim.models.KeyedVectors.load_word2vec_format(embedding_file, binary=True)

  def word2vec(self,tokens):
    # Define the dimensionality (300 dims)
    dim=self.embeddings['word'].size

    # Make a list of embeddings of each word
    word_vec=[]
    for word in tokens:
      if word in self.embeddings:
        word_vec.append(self.embeddings[word])
      # If the word is not present in the embedding, assign a random vector
      else:
        word_vec.append(np.random.uniform(-0.25,0.25,dim))
    return word_vec

  # Function for calculating the similarity of the question vec and target vec.
  def total_similarity(self,vec,ques_vec):
    score = 0
    # Compare all the word vectors in the question to the target vector in terms of cosine similarity
    # Increase of decrease score according to the result
    # Lower cosine distance -> More similar -> Higher score
    for v in ques_vec:
      score += (1 - spatial.distance.cosine(vec, v))
    return score

In [ ]:
# Create an embedding object
emb = word_embedding()

In [ ]:
# Class for reading and processing a question
class question:

  def __init__(self,aline):
      self.fields=aline

  # Get the given field from the question
  def get_field(self,field):
    return self.fields[question.colnames[field]]

  # Get the tokenized question with 'start' and 'end' tokens at each end.
  def get_tokens(self):
    return ["__START"]+tokenize(self.fields[question.colnames["question"]])+["__END"]

  # Add the answer field.
  def add_answer(self,fields):
      self.answer=fields[1]

  # Get window amount of left context words of the blank.
  def get_left_context(self,window=1,target="_____"):
    found=-1
    sent_tokens=self.get_tokens()
    for i,token in enumerate(sent_tokens):
      if token==target:
        found=i
        break  
    if found>-1:
      return sent_tokens[found-window:found]
    else:
      return []

  # Get window amount of the right context words of the blank.
  def get_right_context(self,window=1,target="_____"):
    found=-1
    sent_tokens=self.get_tokens()
    for i,token in enumerate(sent_tokens):
      if token==target:
        found=i
        break  
    if found>-1:
      return sent_tokens[found+1:found+window+1]
    else:
      return []

  # Make a prediction for the question using a given method
  def predict(self,method='word_embedding_total',model=mylm):
    if method=="bigram_backoff":
        return self.choose_backoff(model,methods=["unigram","bigram"],i=1)
    elif method=='trigram_backoff':
      return self.choose_backoff(model,methods=['unigram','bigram','trigram'],i=2)
    elif method=='word_embedding_total':
      return self.embedding_prediction(emb=emb,method='total')
    elif method=='word_embedding_partial':
      return self.embedding_prediction(emb=emb,method='partial')
    else:
      return self.choose(model,method=method)

  # Function for predicting using word2vec similarity
  def embedding_prediction(self,emb,method):
    choices=["a","b","c","d","e"]
    # Tokenize the question (not use the class method since it does not need start and end tokens)
    
    # Create a list of vectors consisting of the question's word vectors
    scores =[]
    candidates = [self.get_field(ch+")") for ch in choices]
    cand_vec = emb.word2vec(candidates)
    
    # If the method is 'total' which compares the choice word vector to all the words in the question
    if method=='total':
      tokens = tokenize(self.fields[question.colnames["question"]])
      ques_vec = emb.word2vec(tokens=tokens)
    # If the method is not total, only compare the choice word to the context words returned with the given window 
    else:
      tokens = self.get_left_context(window=1) + self.get_right_context(window=1)
      ques_vec = emb.word2vec(tokens=tokens)

    # Calculate total similarity of all the choice words
    for word in cand_vec:
      s = emb.total_similarity(word, ques_vec)        
      scores.append(s)
    # Return one with the highest score
    idx = scores.index(max(scores))
    ans = choices[idx]
    return ans
      
  # Make predictions and get scores by comparing to the correct answers
  def predict_and_score(self,method="word_embedding_total",model=mylm):
    prediction=self.predict(method=method,model=model)
    if prediction ==self.answer:
      return 1
    else:
      return 0

  # Calculate probabilities of choice words using the given method and return one with the highest probability.
  def choose(self,lm,method="trigram",choices=[]):
    if choices==[]:
      choices=["a","b","c","d","e"]

    # For unigram, simply get the probability of the choice word.
    if method=='unigram':
      probs=[lm.unigram.get(self.get_field(ch+")"),0) for ch in choices]

    # For bigram, get a context word from the left and right each and calculate probabilities of the word given each context word.
    # Then multiply both probabilties
    elif method=="bigram":
      rc=self.get_right_context(window=1)
      lc=self.get_left_context(window=1)
      probs=[lm.get_prob(rc[0],self.get_field(ch+")"),method)*
             lm.get_prob(self.get_field(ch+")"),lc,method) for ch in choices]

    # For trigram, get two context words from the left and right and calculate trigram probabilities.
    else:
      rc=self.get_right_context(window=2)
      lc=self.get_left_context(window=2)
      probs=[lm.get_prob(rc[1],[self.get_field(ch+")"),rc[0]],method)*
             lm.get_prob(self.get_field(ch+")"),lc,method) for ch in choices]

    # Return one with the highest probability
    maxprob=max(probs)
    bestchoices=[ch for ch,prob in zip(choices,probs) if prob == maxprob]

    # If there are multiple options left, choose one randomly.
    if len(bestchoices)>1:
       print("Randomly choosing from {}".format(len(bestchoices)))
    return np.random.choice(bestchoices)


  # Instead of choosing randomly when there are multiple options, back off to a lower order n-gram.
  # Recursive and controlled with parameter 'i'
  # Starts with the method of the last element of the methods parameter, and back off to the second last one and so on.
  # Higher order methods should be positioned later.
  def choose_backoff(self,lm,methods=['unigram','bigram','trigram'],choices=["a","b","c","d","e"],i=2):

    # Base case: return an answer according to the unigram probability if it couln't be sorted out with bigram model.
    if methods[i] == 'unigram':
      return self.choose(lm,'unigram',choices)

    # Get bigram probabilities
    elif methods[i] == 'bigram':
      rc=self.get_right_context(window=1)
      lc=self.get_left_context(window=1)
      probs=[lm.get_prob(rc[0],self.get_field(ch+")"),'bigram')*
             lm.get_prob(self.get_field(ch+")"),lc,'bigram') for ch in choices]

    # Get trigram probabilities
    elif methods[i] == 'trigram':
      rc=self.get_right_context(window=2)
      lc=self.get_left_context(window=2)
      probs=[lm.get_prob(rc[1],[self.get_field(ch+")"),rc[0]],'trigram')*
            lm.get_prob(self.get_field(ch+")"),lc,'trigram') for ch in choices]

    # Get the best choices and if there are multiple, call this function recursively with the lower order model.
    maxprob=max(probs)
    bestchoices=[ch for ch,prob in zip(choices,probs) if prob == maxprob]
    if len(bestchoices)>1:
      print("Backing off on {} and window {}".format(len(bestchoices),i))
      self.choose_backoff(lm,methods[:i],bestchoices,i-1)
    
    # There will be only one element in bestchoices
    return bestchoices[0]

# Class for processing the questions in files.
class scc_reader:
    
  def __init__(self,qs=questions,ans=answers):
    self.qs=qs
    self.ans=ans
    self.read_files()
      
  def read_files(self):
    # read in the question file
    with open(self.qs) as instream:
      csvreader=csv.reader(instream)
      qlines=list(csvreader)
    
    # store the column names as a reverse index so they can be used to reference parts of the question
    question.colnames={item:i for i,item in enumerate(qlines[0])}
    
    # create a question instance for each line of the file (other than heading line)
    self.questions=[question(qline) for qline in qlines[1:]]
    
    # read in the answer file
    with open(self.ans) as instream:
      csvreader=csv.reader(instream)
      alines=list(csvreader)
        
    # add answers to questions so predictions can be checked    
    for q,aline in zip(self.questions,alines[1:]):
      q.add_answer(aline)
      
  def get_field(self,field):
    return [q.get_field(field) for q in self.questions] 
  
  def predict(self,method="word_embedding_total"):        
    return [q.predict(method=method) for q in self.questions]
    
  def predict_and_score(self,method="word_embedding_total",model=mylm):
    scores=[q.predict_and_score(method=method,model=model) for q in self.questions]
    return sum(scores)/len(scores)
  

In [ ]:
SCC = scc_reader()

In [ ]:
# Train with different numbers of training documents
numbers=[20, 50, 100]
lms = {}
for n in numbers:
  mylm=language_model(trainingdir=trainingdir,files=training[:n],discount=0.3)
  lms[n]=mylm

In [ ]:
results={}
methods=['trigram_backoff','trigram','bigram','bigram_backoff','unigram']
for (n, model) in lms.items():
  results[n]={}
  for method in methods:
    acc=SCC.predict_and_score(method,model)
    results[n][method]=acc

results_pd=pd.DataFrame(results)

In [ ]:
# Plot the perplexity of each model
labels = ['unigram','bigram','trigram']

x = np.arange(len(labels))  # the label locations
width=0.4  # the width of the bars

fig, ax = plt.subplots()
rects1=ax.bar(x - width/2, perplexities['training'].values(), width, label='training')
rects2=ax.bar(x + width/2, perplexities['testing'].values(), width, label='testing')
ax.set_ylabel('Perplexity')
ax.set_title('The perplexity of each model on training and testing data with 100 training documents')
ax.set_xticks(x)
ax.set_xticklabels(methods)
ax.set_ylim((0,800))
ax.legend()

autolabel(rects1)
autolabel(rects2)

plt.show()

In [ ]:
# Calculate accuracy of all the models on the challenge
tri_backoff_sc = SCC.predict_and_score('trigram_backoff')
trigram_sc = SCC.predict_and_score('trigram')
bigram_sc = SCC.predict_and_score('bigram')
bi_backoff_sc=SCC.predict_and_score('bigram_backoff')
unigram_sc=SCC.predict_and_score('unigram')
embedding_total_sc=SCC.predict_and_score('word_embedding_total')
embedding_partial_sc=SCC.predict_and_score('word_embedding_partial')

In [ ]:
embedding_partial_sc_1=SCC.predict_and_score('word_embedding_partial') # compare only a single word from the left and right
print(embedding_partial_sc_1)

0.2778846153846154


In [ ]:
embedding_partial_sc_2=SCC.predict_and_score('word_embedding_partial') # compare two words each form the left and right
print(embedding_partial_sc_2)

0.32019230769230766


In [ ]:
embedding_total_sc=SCC.predict_and_score('word_embedding_total') # compare all the words
print(embedding_total_sc)

0.35865384615384616


In [ ]:
embedding_results=[embedding_partial_sc_1,embedding_partial_sc_2,embedding_total_sc]

In [ ]:
# Plot the accuracy of the distributional similarity method
labels=['First Method', 'Second Method', 'Third Method']

x = np.arange(len(labels))  # the label locations
width=0.4  # the width of the bars

fig, ax = plt.subplots()
rects1=ax.bar(x, embedding_results, width)
ax.set_ylabel('Accuracy')
ax.set_title('The Accuracy of Each Method of Distributional Similarity')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.set_ylim((0,1))

autolabel(rects1)

plt.show()